<a href="https://colab.research.google.com/github/sokanaid/CourseProject2022BiologicalMath/blob/main/model_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение моделей предсказания числинности на плато

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
np.random.seed(0)

In [2]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=aae6d672772cbcc543d406edb254b4cdc0a5b640e8a62e699885223914eb3151
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [3]:
!pip3 install pytorch_lightning torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 20.4 MB/s eta 0:00:00


In [4]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import os
import random
import sys
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from pytorch_lightning.loggers import WandbLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,TensorDataset
from torchmetrics.functional import accuracy
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import PIL
from torchmetrics.functional import accuracy
from wandb.beta.workflows import use_model

In [5]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/sim_tables

Mounted at /content/drive/
/content/drive/MyDrive/sim_tables


In [6]:
def seed_everything(seed):
    # Фискирует максимум сидов.
    # Это понадобится, чтобы сравнение оптимизаторов было корректным
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(123456)

## Подготовка датасета

In [247]:
file_name = "/content/drive/MyDrive/sim_tables/data_set.csv"
sim_frame = pd.read_csv(file_name, sep='\t').drop("Unnamed: 0", axis = 1)

In [248]:
sim_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290525 entries, 0 to 290524
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   b              290525 non-null  float64
 1   d              290525 non-null  float64
 2   death_r        290525 non-null  float64
 3   dd             290525 non-null  float64
 4   sd_b           290525 non-null  float64
 5   sd_d           290525 non-null  float64
 6   area_length_x  290525 non-null  float64
 7   initial_pop    290525 non-null  float64
 8   plateau_pop    290525 non-null  float64
dtypes: float64(9)
memory usage: 19.9 MB


In [249]:
print("total count of simulations:", len(sim_frame))
df = sim_frame[sim_frame["area_length_x"]==100.0].drop("area_length_x", axis = 1)
df1 = df.drop_duplicates(subset=['b', 'd', 'death_r', 'dd', 'sd_b', 'sd_d', 'initial_pop']).reset_index(drop=True)
print("total count of simulations to use in learning:", len(df))

total count of simulations: 290525
total count of simulations to use in learning: 221711


In [250]:
# рапределение размеров в изначальном датасете
sim_frame["area_length_x"].value_counts()

  100.00    221711
1,100.00      3127
3,100.00      2345
2,100.00      2345
4,100.00      2345
2,500.00      2343
3,300.00      2343
3,500.00      2343
2,300.00      2343
  300.00      2343
1,700.00      2343
3,900.00      2343
2,900.00      2343
4,300.00      2343
4,500.00      2343
  500.00      2343
1,900.00      2343
3,700.00      2343
5,100.00      2343
2,700.00      2343
  700.00      2343
  900.00      2343
4,700.00      2343
1,300.00      2343
1,500.00      2343
5,700.00      2342
5,500.00      2342
4,900.00      2342
5,900.00      2342
5,300.00      2342
2,600.00        14
1,600.00        14
3,600.00        14
  600.00        14
4,600.00        13
5,600.00        13
Name: area_length_x, dtype: int64

In [251]:
pd.options.display.float_format = '{:20,.2f}'.format

In [252]:
# распределение параметров
for column in list(df.columns.values):
  print(column)
  print(df[column].describe())

b
count             221,711.00
mean                    0.67
std                     0.23
min                     0.10
25%                     0.55
50%                     0.75
75%                     0.85
max                     0.95
Name: b, dtype: float64
d
count             221,711.00
mean                    0.36
std                     0.22
min                     0.10
25%                     0.20
50%                     0.30
75%                     0.50
max                     0.90
Name: d, dtype: float64
death_r
count             221,711.00
mean                    5.36
std                     2.96
min                     0.10
25%                     3.00
50%                     5.00
75%                     8.00
max                    10.00
Name: death_r, dtype: float64
dd
count             221,711.00
mean                    0.55
std                     0.29
min                     0.10
25%                     0.30
50%                     0.55
75%                     0.80
max     

In [253]:
df["plateau_pop"].describe()

count             221,711.00
mean                  134.51
std                   757.02
min                     0.00
25%                     0.00
50%                     0.00
75%                     0.00
max                34,227.56
Name: plateau_pop, dtype: float64

In [254]:
X = df.drop("plateau_pop", axis =1)
y = df["plateau_pop"]

In [255]:
X.head()

,b,d,death_r,dd,sd_b,sd_d,initial_pop
0,0.35,0.10,3.00,0.90,0.50,0.65,100.00
1,0.45,0.30,7.00,0.70,0.95,0.80,1.00
2,0.95,0.80,9.00,0.20,0.80,0.95,1.00
3,0.35,0.20,1.00,0.60,0.80,0.65,1.00
4,0.35,0.20,6.00,0.10,0.65,0.65,1.00


In [256]:
y.head()

0                   0.00
1                   0.00
2                   0.00
3                   0.00
4                   0.00
Name: plateau_pop, dtype: float64

In [257]:
X_train, X_t, y_train, y_t = train_test_split(X, y, test_size=0.3, random_state = 3)
X_train

,b,d,death_r,dd,sd_b,sd_d,initial_pop
91131,0.85,0.80,6.00,0.90,0.80,0.80,100.00
136496,0.35,0.20,3.00,0.40,0.80,0.50,50.00
55046,0.95,0.20,6.00,0.10,0.50,0.95,50.00
9357,0.35,0.20,9.00,0.20,0.80,0.95,50.00
202309,0.75,0.30,9.00,0.90,0.95,0.80,50.00
...,...,...,...,...,...,...,...
62976,0.75,0.60,1.00,0.30,0.80,0.95,1.00
249107,0.95,0.10,6.00,0.10,0.50,0.95,1.00
101002,0.35,0.20,2.00,0.70,0.95,0.80,100.00
259791,0.75,0.20,6.00,0.30,0.50,0.50,50.00


In [258]:
y_t

52386                    0.00
207012                 130.25
132679                   0.00
273920                   0.00
266077                 417.72
                 ...         
237742                   0.00
179741                   0.00
158214                 367.64
13949                  283.65
105356                   0.00
Name: plateau_pop, Length: 66514, dtype: float64

In [259]:
X_test,X_val, y_test, y_val = train_test_split(X_t, pd.DataFrame(y_t), test_size=0.4, random_state = 3)
X_val

,b,d,death_r,dd,sd_b,sd_d,initial_pop
108079,0.95,0.50,9.00,0.60,0.95,0.80,100.00
88856,0.75,0.40,1.00,0.50,0.65,0.95,100.00
184608,0.75,0.70,8.00,0.70,0.50,0.95,1.00
142507,0.35,0.10,0.10,0.90,0.50,0.95,1.00
56300,0.75,0.40,7.00,0.10,0.50,0.95,1.00
...,...,...,...,...,...,...,...
237398,0.55,0.30,10.00,0.20,0.50,0.80,100.00
192025,0.25,0.10,1.00,0.80,0.95,0.95,100.00
159130,0.85,0.20,5.00,0.30,0.80,0.95,50.00
225534,0.35,0.20,7.00,0.60,0.95,0.50,100.00


In [260]:
y_test

,plateau_pop
254659,0.00
240613,0.00
282996,0.00
74306,0.00
223199,0.00
...,...
244080,492.41
97456,0.00
124917,0.00
110641,0.00


In [261]:
y_test = np.array(y_test)
y_test

array([[8.29650606e-112],
       [2.09875125e-091],
       [8.29650606e-112],
       ...,
       [6.84857036e-101],
       [2.49950348e-105],
       [2.00888470e-109]])

In [262]:
y_val = np.array(y_val)
y_val

array([[9.16964044e-103],
       [3.02924846e+002],
       [1.21304703e-111],
       ...,
       [2.03050070e+002],
       [2.45024669e-109],
       [5.42278710e-056]])

In [263]:
y_train = np.array(y_train)
y_train

array([8.96424855e-109, 1.31499617e-097, 6.21268462e+002, ...,
       5.21862371e-104, 1.53279122e-076, 4.95610755e-101])

In [264]:
y_train

array([8.96424855e-109, 1.31499617e-097, 6.21268462e+002, ...,
       5.21862371e-104, 1.53279122e-076, 4.95610755e-101])

In [265]:
print("train len", len(y_train))
print("test len ", len(y_t))
print("val len ", len(y_val))


train len 155197
test len  66514
val len  26606


In [266]:
# нормализация данных
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_val = pd.DataFrame(scaler.transform(X_val), columns = X_val.columns)
X_train

,b,d,death_r,dd,sd_b,sd_d,initial_pop
0,0.77,1.98,0.22,1.22,0.45,0.45,1.25
1,-1.42,-0.73,-0.80,-0.52,0.45,-1.34,0.02
2,1.21,-0.73,0.22,-1.56,-1.34,1.34,0.02
3,-1.42,-0.73,1.23,-1.22,0.45,1.34,0.02
4,0.33,-0.28,1.23,1.22,1.34,0.45,0.02
...,...,...,...,...,...,...,...
155192,0.33,1.07,-1.47,-0.87,0.45,1.34,-1.18
155193,1.21,-1.19,0.22,-1.56,-1.34,1.34,-1.18
155194,-1.42,-0.73,-1.14,0.52,1.34,0.45,1.25
155195,0.33,-0.73,0.22,-0.87,-1.34,-1.34,0.02


In [267]:
y_train = np.reshape(y_train, (y_train.shape[0],1 ))
y_train

array([[8.96424855e-109],
       [1.31499617e-097],
       [6.21268462e+002],
       ...,
       [5.21862371e-104],
       [1.53279122e-076],
       [4.95610755e-101]])

In [268]:
scaler_y = StandardScaler()
scaler_y.fit(y_train)
y_train = scaler_y.transform(y_train)
y_test = scaler_y.transform(y_test)
y_val = scaler_y.transform(y_val)
y_train

array([[-0.17736242],
       [-0.17736242],
       [ 0.64752227],
       ...,
       [-0.17736242],
       [-0.17736242],
       [-0.17736242]])

## Обучение моделей

### Линейные регрессии

In [167]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()
model = model.fit(X_train, y_train)
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)
print("MSE на тренировочной выборке:")
print(mean_squared_error(scaler_y.inverse_transform(y_train), scaler_y.inverse_transform(np.reshape(pred_train, (pred_train.shape[0],1 )))))
print("MSE на тестовой выборке:")
print(mean_squared_error(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(np.reshape(pred_test, (pred_test.shape[0],1 )))))
model.coef_

0.09731184782514135
0.09932698616161295
MSE на тренировочной выборке:
512047.05769567395
MSE на тестовой выборке:
520263.5109154538


array([[ 0.12849659, -0.18237533, -0.21857941, -0.14586917,  0.00920277,
        -0.00915099, -0.05258099]])

In [168]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=10)
clf.fit(X_train,y_train)
print('a:',clf.intercept_)
print( "b: ",pd.DataFrame({"name":X_train.columns.values, "coef":clf.coef_}))
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))


a: [1.08048535e-17]
b:            name                 coef
0            b                 0.00
1            d                -0.00
2      death_r                -0.00
3           dd                -0.00
4         sd_b                 0.00
5         sd_d                -0.00
6  initial_pop                -0.00
0.0
-3.423290443627103e-05


In [169]:
from sklearn.linear_model import Ridge
import numpy as np
clf1 = Ridge(alpha=10)
clf1.fit(X_train,y_train)
print('a:',clf1.intercept_)
print(clf1.score(X_train, y_train))
print(clf1.score(X_test, y_test))
print("MSE на тренировочной выборке:")
print(mean_squared_error(scaler_y.inverse_transform(y_train), scaler_y.inverse_transform(clf1.predict(X_train))))
print("MSE на тестовой выборке:")
print(mean_squared_error(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(clf1.predict(X_test))))
clf1.coef_

a: [-3.40308811e-17]
0.09731184709113083
0.09932680093682311
MSE на тренировочной выборке:
512047.0581120392
MSE на тестовой выборке:
520263.61790844455


array([[ 0.1284759 , -0.18235349, -0.21856473, -0.14585975,  0.00920224,
        -0.00915045, -0.05257793]])

### Решающие деревья

In [175]:
X_train, X_t, y_train, y_t = train_test_split(X, y, test_size=0.3, random_state = 3)
X_test,X_val, y_test, y_val = train_test_split(X_t, y_t, test_size=0.4, random_state = 3)
y_test = np.array(y_test)
y_val = np.array(y_val)
y_train = np.array(y_train)
y_train = np.reshape(y_train, (y_train.shape[0],1 ))
y_val = np.reshape(y_val, (y_val.shape[0],1 ))
y_test = np.reshape(y_test, (y_test.shape[0],1 ))

In [176]:
X_test

,b,d,death_r,dd,sd_b,sd_d,initial_pop
254659,0.95,0.70,1.00,0.10,0.50,0.65,1.00
240613,0.95,0.40,7.00,0.30,0.50,0.65,100.00
282996,0.65,0.50,5.00,0.80,0.95,0.65,1.00
74306,0.95,0.40,7.00,0.50,0.95,0.65,50.00
223199,0.85,0.40,9.00,1.00,0.80,0.65,1.00
...,...,...,...,...,...,...,...
244080,0.95,0.20,1.00,0.90,0.50,0.65,1.00
97456,0.65,0.50,1.00,0.60,0.65,0.65,100.00
124917,0.45,0.10,7.00,0.60,0.80,0.95,50.00
110641,0.85,0.80,1.00,0.80,0.95,0.95,50.00


In [177]:
y_val

array([[9.16964044e-103],
       [3.02924846e+002],
       [1.21304703e-111],
       ...,
       [2.03050070e+002],
       [2.45024669e-109],
       [5.42278710e-056]])

In [178]:
y_train

array([[8.96424855e-109],
       [1.31499617e-097],
       [6.21268462e+002],
       ...,
       [5.21862371e-104],
       [1.53279122e-076],
       [4.95610755e-101]])

In [182]:
from sklearn.ensemble import RandomForestRegressor
best_mse = 10000000000
best = [0,0]
for max_depth in [3, 5, 10, 20]:
  for min_samples_leaf in [3, 5, 10, 15]:
    rf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = max_depth, min_samples_leaf=min_samples_leaf, random_state = 18)
    rf.fit(X_train, y_train)
    print('max_depth', max_depth, "min_samples_leaf", min_samples_leaf)
    print(rf.score(X_train, y_train))
    print(rf.score(X_test, y_test))
    print("MSE на тренировочной выборке:")
    print(mean_squared_error(y_train, rf.predict(X_train)))
    print("MSE на тестовой выборке:")
    mse = mean_squared_error(y_test, rf.predict(X_test))
    print(mse)
    print()
    if mse < best_mse:
      best_mse = mse
      best = [ max_depth,min_samples_leaf]

<ipython-input-182-1fe4c8f4a000>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


max_depth 3 min_samples_leaf 3
0.3066628716691213
0.3039288289787955
MSE на тренировочной выборке:
393293.33801228815
MSE на тестовой выборке:
402077.5861144032



<ipython-input-182-1fe4c8f4a000>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


max_depth 3 min_samples_leaf 5
0.30594001838388785
0.30379014253855874
MSE на тренировочной выборке:
393703.3742988593
MSE на тестовой выборке:
402157.6967574505



<ipython-input-182-1fe4c8f4a000>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


...

max_depth 20 min_samples_leaf 5
0.9184605949835604
0.8937530062014084
MSE на тренировочной выборке:
46252.97487768075
MSE на тестовой выборке:
61372.36618458989



<ipython-input-182-1fe4c8f4a000>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


max_depth 20 min_samples_leaf 10
0.8664701593346364
0.8486973114884162
MSE на тренировочной выборке:
75744.38842754088
MSE на тестовой выборке:
87398.2752081307



<ipython-input-182-1fe4c8f4a000>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


max_depth 20 min_samples_leaf 15
0.8369898846519473
0.8236550451416418
MSE на тренировочной выборке:
92466.98290821722
MSE на тестовой выборке:
101863.6552191616



In [183]:
print()
print("лучшее MSE на тестовой выборке:", best_mse, "max_depth", best[0], "min_samples_leaf", best[1] )


лучшее MSE на тестовой выборке: 49598.083186192525 max_depth 20 min_samples_leaf 3


In [186]:
from sklearn.ensemble import BaggingRegressor
for n_estimators in [50, 100, 200, 500, 1000]:
  print("n_estimators", n_estimators)
  br = BaggingRegressor(n_estimators = n_estimators, random_state = 18)
  br.fit(X_train, y_train)
  print(br.score(X_train, y_train))
  print(br.score(X_test, y_test))
  print("MSE на тренировочной выборке:")
  print(mean_squared_error(y_train, br.predict(X_train)))
  print("MSE на тестовой выборке:")
  print(mean_squared_error(y_test, br.predict(X_test)))

n_estimators 50


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9897095305240793
0.9237931640123339
MSE на тренировочной выборке:
5837.236929228721
MSE на тестовой выборке:
44020.01107786659
n_estimators 100


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9900183852975766
0.9252881210017915
MSE на тренировочной выборке:
5662.0400158279035
MSE на тестовой выборке:
43156.46619525917
n_estimators 200


...

0.9903681160042604
0.9257551337785471
MSE на тренировочной выборке:
5463.656355965037
MSE на тестовой выборке:
42886.70158241499
n_estimators 500


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9904998341321266
0.9264905642438858
MSE на тренировочной выборке:
5388.939656009943
MSE на тестовой выборке:
42461.88854810339
n_estimators 1000


/usr/local/lib/python3.9/dist-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9905593322628081
0.9271069883499121
MSE на тренировочной выборке:
5355.189525712534
MSE на тестовой выборке:
42105.81818218063


### обучение нейронной модели

In [269]:
X_train, X_t, y_train, y_t = train_test_split(X, y, test_size=0.3, random_state = 3)
X_test,X_val, y_test, y_val = train_test_split(X_t, y_t, test_size=0.4, random_state = 3)
y_test = np.array(y_test)
y_val = np.array(y_val)
y_train = np.array(y_train)
y_train = np.reshape(y_train, (y_train.shape[0],1 ))
y_val = np.reshape(y_val, (y_val.shape[0],1 ))
y_test = np.reshape(y_test, (y_test.shape[0],1 ))

In [292]:
class YourModule(pl.LightningModule):
    def __init__(self , learning_rate = 0.01):
        super().__init__()
        import torch
        torch.cuda.empty_cache()
      
        self.loss_func = F.mse_loss

        self.model = nn.Sequential(nn.Linear(in_features=7, out_features=200, bias=True), 
                                  nn.ReLU(), 
                                  nn.Linear(in_features=200, out_features=100, bias=True),
                                  nn.ReLU(),
                                  nn.Linear(in_features=100, out_features=1, bias=True),)
        self.learning_rate = learning_rate
        print('lr1',self.learning_rate)
        
    def configure_optimizers(self):
        print("configure_optimizers")
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        print('lr',self.learning_rate)
        #optimizer =  torch.optim.SGD(self.model.parameters(), lr=self.learning_rate, momentum=0.9)

        # Decay LR by a factor of 0.1 every 7 epochs
        exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x, target = train_batch
        loss = self.loss_func(self.model(x), target)
        #loss.backward()
        self.log("train_loss MSE", loss, prog_bar=True)  # сохраняет логи в папку, но можно несложно добавить wandb
        return loss

    def validation_step(self, val_batch, batch_idx):
        with torch.no_grad():
          x, target = val_batch
          loss =  self.loss_func(self.model(x), target)
          self.log("val_loss MSE", loss, prog_bar=True)

In [271]:
batch_size = 32
train_dataloader = DataLoader(TensorDataset(torch.tensor(X_train.values.astype(np.float32)), torch.Tensor(y_train)), batch_size=batch_size,
    shuffle=True,  
    num_workers=1)
test_dataloader = DataLoader(TensorDataset(torch.tensor(X_test.values.astype(np.float32)), torch.Tensor(y_test)), batch_size=batch_size,
    shuffle= False, 
    num_workers=1)

In [272]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [273]:
wandb_logger = WandbLogger(log_model='all')
# какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [274]:
for x, y in train_dataloader:
  print(x, y)
  break

tensor([[  0.2500,   0.1000,   7.0000,   0.5000,   0.5000,   0.6500, 100.0000],
        [  0.9500,   0.4000,   6.0000,   0.7000,   0.9500,   0.5000,  50.0000],
        [  0.6500,   0.2000,   6.0000,   0.6000,   0.9500,   0.5000,   1.0000],
        [  0.6500,   0.1000,   2.0000,   0.9000,   0.5000,   0.5000, 100.0000],
        [  0.2500,   0.1000,   7.0000,   0.5000,   0.9500,   0.6500,  50.0000],
        [  0.5500,   0.5000,   3.0000,   0.6000,   0.9500,   0.8000,  50.0000],
        [  0.9500,   0.5000,   2.0000,   0.2000,   0.6500,   0.5000, 100.0000],
        [  0.3500,   0.2000,   2.0000,   0.2000,   0.5000,   0.6500,  50.0000],
        [  0.8500,   0.2000,   1.0000,   0.9000,   0.5000,   0.8000,   1.0000],
        [  0.8500,   0.1000,   4.0000,   0.7000,   0.6500,   0.6500,  50.0000],
        [  0.4500,   0.1000,   4.0000,   0.2000,   0.8000,   0.5000,  50.0000],
        [  0.5500,   0.3000,   3.0000,   0.9000,   0.9500,   0.9500,   1.0000],
        [  0.9500,   0.8000,   5.0000,  

In [293]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
module = YourModule()
f = pl.Trainer(logger=wandb_logger, accelerator="gpu", max_epochs=10)
               #, auto_lr_find=True)

#f.tune(module, train_dataloader, test_dataloader)
torch.set_grad_enabled(True)
f.fit(module, train_dataloader, test_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 21.8 K
-------------------------------------
21.8 K    Trainable params
0         Non-trainable params
21.8 K    Total params
0.087     Total estimated model params size (MB)


lr1 0.01
configure_optimizers
lr 0.01


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
